# Develop a vcf annotater that will translate SNPs into amino acid changes

May 1, 2018 

I was having trouble with SNPEff mis-annotating coding region changes in M2 and NEP, so I decided to write my own. This is a simple notebook that will read in the reference sequences that reads were mapped to, a gtf file for those references sequences with coding region coordinates, and a vcf file output by Varscan. It will then determine whether SNVs occur within a coding region, and if so, annotate as amino acid changing or not. Currently, this will not annotate SNVs that fall outside of a coding region. 

In [1]:
import sys, subprocess, glob, os, shutil, re, importlib, csv
from subprocess import call
import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio import Seq
from Bio import AlignIO
from Bio.Seq import Seq

In [2]:
# dictionary convert single letter to 3 letter amino acid symbols
amino_acid_abbreviations = {"A": "Ala","R": "Arg","N": "Asn","D": "Asp","C": "Cys","Q": "Gln","E": "Glu","G": "Gly","H": "His","I": "Ile","L": "Leu","K": "Lys","M": "Met","F": "Phe","P": "Pro","O": "Pyl","S": "Ser","U": "Sec","T": "Thr","W": "Trp","Y": "Tyr","V": "Val", "B":"Asx","Z":"Glx","U":"Sec","X":"Xaa","J":"Xle","*":"Stop"}

In [3]:
# read in reference sequences and gtfs and store information about protein sequences

def define_coding_regions(gtf_file):
    with open(gtf_file, "r") as gtf:
        coding_regions = {}
        
        for line in gtf:
            if line.strip("\n") != "":      # ignore blank lines (otherwise throws an index error)
                sequence_name = line.split("\t")[0]
                annotation_type = line.split("\t")[2]
                start = int(line.split("\t")[3]) - 1  # adding the -1 here for 0 indexing
                stop = int(line.split("\t")[4]) - 1    # adding the -1 here for 0 indexing
                gene_name = line.split("\t")[8]
                gene_name = gene_name.split(";")[0]
                gene_name = gene_name.replace("gene_id ","")
                gene_name = gene_name.replace("\"","")
            
                if annotation_type.lower() == "cds":
                    if sequence_name not in coding_regions:
                        coding_regions[sequence_name] = {}
                        coding_regions[sequence_name][gene_name] = [start, stop]
                    elif sequence_name in coding_regions and gene_name not in coding_regions[sequence_name]:
                        coding_regions[sequence_name][gene_name] = [start, stop]
                    elif gene_name in coding_regions[sequence_name]:
                        coding_regions[sequence_name][gene_name].extend([start, stop])
        
            # sort coding region coordinates so that they are always in the correct order
            for sequence_name in coding_regions:
                for gene in coding_regions[sequence_name]:
                    coding_regions[sequence_name][gene] = sorted(coding_regions[sequence_name][gene])
                    
    return(coding_regions)

In [4]:
# pull in fasta file and store gene segments in a dictionary
def read_fastas(fasta_file):
    reference_sequence = {}
    
    for seq in SeqIO.parse(fasta_file, "fasta"):
        sequence_name = str(seq.id)
        sequence = str(seq.seq).lower()
        reference_sequence[sequence_name] = sequence
    
    return(reference_sequence)

In [5]:
# divide up sequences into their coding regions 
def slice_fastas(coding_regions, reference_sequence, fasta_file):
    transcripts = {}
    stop_codons = ["taa","tag","tga"]
    refseqname = fasta_file.split("/")[7]
    
    for c in coding_regions: 
        for gene in coding_regions[c]:
            transcripts[gene] = ""
            coordinates = coding_regions[c][gene]   # define the coding regions for each gene
            
            for i in range(0,int(len(coordinates)),2):   # go along the coordinates in chunks of 2 at a time
                sequence_chunk = reference_sequence[c][coordinates[i]:coordinates[i+1]+1]
                transcripts[gene] += sequence_chunk     # append each piece of the transcript together 
    
    # loop through each transcript to make sure that it begins with a start codon and ends with a stop codon
    for t in transcripts: 
        if transcripts[t][0:3] != "atg":
            print("WARNING! " + refseqname + " " + t + "does not contain a start codon!")
        if transcripts[t][-3:] not in stop_codons:
            print("WARNING! " + refseqname + " " + t + " does not contain a stop codon! These are the last 3 nucleotides: " + transcripts[t][-3:])
            
    #print(transcripts)
    return(transcripts)

In [6]:
# open vcf, read in SNPs, and annotate with amino acid changes 

def annotate_amino_acid_changes(coding_regions, reference_sequence, transcripts, vcf, aa_abreviations):
    
    with open(vcf, "r") as csvfile:
        outfilename = vcf.replace(".vcf",".LHM-annotated-variants.txt")
        
        with open(outfilename, "w") as outfile:
            to_write = ["sample","gene","reference_position","reference_allele","variant_allele","coding_region_change","synonymous/nonsynonymous","frequency(%)","frequency","\n"]
            to_write2 = "\t".join(to_write)
            outfile.write(to_write2)
        
        reader = csv.reader(csvfile, delimiter="\t")    
        for row in reader:
            
            # ignore comment lines
            if "##" not in row[0] and "#CHROM" not in row[0]:
                
                sequence_name = row[0]
                site = int(row[1]) - 1     # to make this 0 indexed
                reference_allele = row[3].lower()
                alternative_allele = row[4].lower()
                
                # pull out the frequency using a string search
                SearchStr = '.+\:([0-9]{1,2}\.{0,1}[0-9]{0,2}\%)'
                result = re.search(SearchStr,row[9])
                if result:
                    frequency = '\t'.join(result.groups())
                    frequency_decimal = frequency.replace("%","")
                    frequency_decimal = (float(frequency_decimal))/100
                    frequency_decimal = "%.4f" % frequency_decimal    # only include 4 numbers after decimal

                else:
                    frequency = "none reported"

                # figure out whether the SNP lies within a coding region: 
                for gene in coding_regions[sequence_name]:
                    coordinates = coding_regions[sequence_name][gene]

                    # go through gene coordinates 2 at a time; this is for genes with multiple regions 
                    for i in range(0,int(len(coordinates)),2):
                        if site >= coordinates[i] and site <= coordinates[i+1]:  # if site is within the gene
                            
                            # determine the coding region site, depending on if there are 2 frames or 1 
                            if len(coordinates) == 2:
                                cds_site = site - coordinates[i]
                            elif len(coordinates) == 4:
                                cds_site = (coordinates[1] - coordinates[0]) + (site - coordinates[2] + 1)
                                #print(gene,coordinates,site,cds_site)
                                
                            # now determine whether the site is in the 1st, 2nd, or 3rd codon position
                            # if SNP is in 1st position in codon:
                            aa_site = int(cds_site/3)+1
                            
                            if float(cds_site) % 3 == 0:
                                codon = transcripts[gene][cds_site:cds_site+3]
                                variant_codon = alternative_allele + codon[1:3]
                                variant_aa = Seq(variant_codon).translate()
                                ref_codon = reference_allele + codon[1:3]
                                ref_aa = Seq(ref_codon).translate()
                            
                            # if variant is in the middle of the codon:    
                            elif float(cds_site - 1) % 3 == 0:
                                codon = transcripts[gene][cds_site-1:cds_site+2]
                                variant_codon = codon[0] + alternative_allele + codon[2]
                                variant_aa = Seq(variant_codon).translate()
                                ref_codon = codon[0] + reference_allele + codon[2]
                                ref_aa = Seq(ref_codon).translate()
                            
                            # if the variant is in the 3rd codon position
                            elif float(cds_site -2) % 3 == 0:
                                codon = transcripts[gene][cds_site-2:cds_site+1]
                                variant_codon = codon[0:2] + alternative_allele
                                variant_aa = Seq(variant_codon).translate()
                                ref_codon = codon[0:2] + reference_allele
                                ref_aa = Seq(ref_codon).translate()
                                
                                
                            # return the amino acid changes, with single letters converted to 3-letter aa codes
                            if ref_aa == variant_aa:
                                syn_nonsyn = "synonymous"
                            elif ref_aa != variant_aa and variant_aa == "*":
                                syn_nonsyn = "stop_gained"
                            elif ref_aa != variant_aa:
                                syn_nonsyn = "nonsynonymous"
                            
                            amino_acid_change = amino_acid_abbreviations[ref_aa] + str(aa_site) + amino_acid_abbreviations[variant_aa]
                        
                        #else: 
                            #amino_acid_change = "outside coding region"
                            #syn_nonsyn = "outside coding region"
                            
                                                
                         
                            with open(outfilename, "a") as outfile: 
                                output = [sequence_name,gene,str(site+1),reference_allele.upper(),alternative_allele.upper(),amino_acid_change,syn_nonsyn,frequency,frequency_decimal,"\n"]
                                output2 = "\t".join(output)
                                outfile.write(output2)


In [7]:
# read in vcfs, gtfs, and fasta files into a single dictionary based on sampleid

vcfs_directory = "/Volumes/gradschool-and-postdoc-backups/post-doc/stored_files_too_big_for_laptop/H5N1_Cambodia_outbreak_study/Cambodia_H5_sequence_raw_data/combined_human_and_bird_usable_subset/"
gtfs_directory = "/Volumes/gradschool-and-postdoc-backups/post-doc/stored_files_too_big_for_laptop/H5N1_Cambodia_outbreak_study/Cambodia_H5_sequence_raw_data/combined_human_and_bird_usable_subset/combined_vcfs_nodups/gtfs/"
sampleids = {}

for f in glob.glob(vcfs_directory + "*/coverage_norm_and_duplicate_read_removal/*.varscan0.01.vcf"):  #coverage_norm_and_duplicate_read_removal/
    sampleid = f.replace(vcfs_directory, "")
    sampleid = sampleid.split("/")[2]
    sampleid = "_".join(sampleid.split("_")[1:-3])
    sampleid = sampleid.replace("_ori2","")
    sampleid = sampleid.replace("CAMBODIA","Cambodia")
    sampleid = sampleid.replace("Combodia","Cambodia")
    
    sampleids[sampleid] = {}
    sampleids[sampleid]["vcf"] = f

for f in glob.glob(gtfs_directory + "*"):
    sampleid = f.replace(gtfs_directory, "")
    sampleid = "_".join(sampleid.split("_")[1:])
    sampleid = sampleid.replace("_ori2","")
    
    if sampleid != "duck_Guangdong_173_2004" and sampleid != "I_didnt_use":
        for g in glob.glob(f + "*/genes.gtf"):
            sampleids[sampleid]["gtf"] = g
        for h in glob.glob(f + "*/sequences.fa"):
            sampleids[sampleid]["fasta"] = h
 

In [9]:
#sampleids

In [12]:
# run it!
for s in sampleids:
    coding_regions = define_coding_regions(sampleids[s]["gtf"])
    reference_sequence = read_fastas(sampleids[s]["fasta"])
    transcripts = slice_fastas(coding_regions, reference_sequence, sampleids[s]['fasta'])
    vcf = sampleids[s]["vcf"]
    annotate_amino_acid_changes(coding_regions, reference_sequence, transcripts, vcf, amino_acid_abbreviations)

/Users/lmoncla/anaconda/lib/python3.6/site-packages/Bio/Seq.py:150: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  "the new string hashing behaviour.", BiopythonWarning)


WARNING! combined_human_and_bird_usable_subset HA does not contain a stop codon! These are the last 3 nucleotides: tgc


In [13]:
# I ran this from the terminal to combine everything

# for f in */coverage_norm_and_duplicate_read_removal/*.LHM-annotated-variants.txt; do cp $f /Users/lmoncla/Documents/H5N1_Cambodian_outbreak_study/Cambodia_H5_sequence_raw_data/combined_human_and_bird_usable_subset/combined_vcfs_nodups; done

In [14]:
# combine all vcfs in this folder together into 1 parseable file
output_directory = "/Volumes/gradschool-and-postdoc-backups/post-doc/stored_files_too_big_for_laptop/H5N1_Cambodia_outbreak_study/Cambodia_H5_sequence_raw_data/combined_human_and_bird_usable_subset/combined_vcfs_nodups/"
vcf_outs = []
outfilename = output_directory + "combined_variants_nodups_2019-06-03.txt"

for f in glob.glob(output_directory + "*.txt"):
    vcf_outs.append(f)
    
with open(outfilename, "w") as outfile:
    header = ["sampleid","sample","gene","reference_position","reference_allele","variant_allele","coding_region_change","synonymous_nonsynonymous","frequency(%)","frequency","\n"]
    header = "\t".join(header)
    outfile.write(header)
    
    for v in vcf_outs:
        with open(v, "r") as infile:
            for line in infile:
                if "sample" not in line:
                    sample = "/".join(line.split("\t")[0].split("_")[1:-1])
                    sampleid = line.split("\t")[0]
                    rest = "\t".join(line.split("\t")[1:])
                    outfile.write(sampleid+"\t"+sample+"\t"+rest)